In [1]:
import rqdatac
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import FamaMacBeth
import csv
import numpy as np
import import_ipynb
import a_general_factor_test
import c_index_factor_loader
rqdatac.init()
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
factor_name = '第二产业增加值占GDP比重(现价)'
remaining_time_to_mature =['短期 (1-3年)', '中期 (3-5年)', '长期 (5+年)']
rolling_window = 90


'''

000001.XSHG	上证综合指数
000300.XSHG	沪深300指数
000905.XSHG	中证500指数
000852.XSHG	中证1000指数
399001.XSHE	深证成份指数
399006.XSHE	创业板指
000688.XSHG	科创50

'''

index_list = ['000001.XSHG',
'000300.XSHG',
'000905.XSHG',
'000852.XSHG',
'399006.XSHE',
'000688.XSHG']

/opt/anaconda3/envs/rqsdk/lib/python3.9/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  28 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [2]:
def main_test_for_index_get_factors(start_date,end_date,instrument_type,factor_name,remaining_time_to_mature,rolling_window,folder_name):
    import os
    success_file_path,dropped_file_path,error_filepath = a_general_factor_test.generate_file_path(folder_name=folder_name)
    try:
        
        all_instruments = a_general_factor_test.get_bonds_poll(start_date, end_date, instrument_type)
        dependent = a_general_factor_test.get_return_rate(all_instruments,start_date,end_date)
        independent = c_index_factor_loader.get_profolio_factor(factor_name,start_date,end_date)
        time_regression_df = a_general_factor_test.prepare_time_regression_df(dependent,independent)
        rolling_betas = a_general_factor_test.roll_data_and_regress(time_regression_df, rolling_window)
        #cb only
        rolling_betas_with_timegroup_tag = a_general_factor_test.add_maturity_group_tag(rolling_betas,all_instruments)
        rolling_betas = a_general_factor_test.select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature)
        result = a_general_factor_test.do_fama_macbeth(rolling_betas,dependent)
        a_general_factor_test.store_value_into_file(results=result,factor_name=factor_name,success_filepath=success_file_path,dropped_filepath=dropped_file_path,error_filepath=error_filepath)
        
    except Exception as e:
        data = f'factor_name:  {factor_name}    outside error\n'
        with open(error_filepath,'a') as file:
                        file.write(data)
        print(e)

In [3]:


for mature_remain in remaining_time_to_mature:
    folder_name = f'index_{mature_remain}_factor'
    for factor_name in index_list:
        main_test_for_index_get_factors(start_date=start_date
                                                             ,end_date= end_date
                                                             ,instrument_type=instrument_type
                                                             ,factor_name = factor_name
                                                             ,remaining_time_to_mature = mature_remain
                                                             ,rolling_window =rolling_window,
                                                             folder_name = folder_name)

testmodel2
可转债数量: 449
               value
date                
2018-01-03  0.006207
2018-01-04  0.004928
2018-01-05  0.001784
2018-01-08  0.005227
2018-01-09  0.001296
...              ...
2020-12-25  0.009946
2020-12-28  0.000213
2020-12-29 -0.005372
2020-12-30  0.010481
2020-12-31  0.017167

[729 rows x 1 columns]
                             FamaMacBeth Estimation Summary                            
Dep. Variable:                     return_30   R-squared:                        0.0010
Estimator:                       FamaMacBeth   R-squared (Between):              0.0065
No. Observations:                      48260   R-squared (Within):              -0.0050
Date:                       Fri, Jul 18 2025   R-squared (Overall):              0.0010
Time:                               17:09:22   Log-likelihood                 4.226e+04
Cov. Estimator:    Fama-MacBeth Standard Cov                                           
                                               F-statistic:      